## Natural Language Processing with Disaster Tweets

## Predict which Tweets are about real disasters and which ones are not

Twitter has become an important communication channel in times of emergency. The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter.

We have access to a dataset of 10,000 tweets that were hand classified.

### Import libraries

In [42]:
!python -m spacy download en_core_web_sm -q

     |████████████████████████████████| 12.0 MB 5.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
import json
import pathlib
import os
import io
import warnings
import en_core_web_sm
import plotly.graph_objects as go

from google.colab import drive
from spacy.lang.en.stop_words import STOP_WORDS
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from plotly.subplots import make_subplots
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')

### Import data


In [44]:
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/src/train.csv', encoding="utf-8")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Data preprocessing

In [45]:
# let's take the columns we're interested in
df = df[["text", "target"]]
df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [46]:
# DO NOT RUN THIS COMMAND (TAKES TIME)
# IMPORT INSTEAD THE CLEANED DATASET IN THE NEXT CELL

# spacy and english initialisation
nlp = en_core_web_sm.load()

# clean up the character strings and encode the words so they are represented as integers
df["text_clean"] = df["text"].apply(lambda x: "".join(ch for ch in x if ch.isalnum() or ch == " "))
df["text_clean"] = df["text_clean"].apply(lambda x: x.replace(" +", " ").lower().strip())
df["text_clean"] = df["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

df = df[df["text_clean"].notna()]
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/src/train_clean.csv", index=False)
df

,text,target,text_clean
0,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake allah forgive
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfire evacuation order...
4,Just got sent this photo from Ruby #Alaska as ...,1,send photo ruby alaska smoke wildfire pour school
...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,1,giant crane hold bridge collapse nearby home h...
7609,@aria_ahrary @TheTawniest The out of control w...,1,ariaahrary thetawniest control wild fire calif...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m194 0104 utc5 km s volcano hawaii httptcozdto...
7611,Police investigating after an e-bike collided ...,1,police investigate ebike collide car little po...


In [47]:
# import the cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/src/train_clean.csv', encoding="utf-8")

In [48]:
vocab_size = 10000

# using tf.keras.preprocessing.text.Tokenizer process to encoding all the reviews
# instanciate the tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df["text_clean"])

df["text_encoded"] = tokenizer.texts_to_sequences(df.text_clean)
df["len_text"] = df["text_encoded"].apply(lambda x: len(x))
df = df[df["len_text"] != 0]
df.head()

,text,target,text_clean,text_encoded,len_text
0,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake allah forgive,"[3679, 410, 167, 1368, 1944]",5
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[112, 2, 156, 504, 5569, 5570, 955]",7
2,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...,"[1369, 442, 1692, 318, 5571, 288, 182, 1692, 3...",11
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfire evacuation order...,"[2312, 6, 2313, 68, 182, 289, 34]",7
4,Just got sent this photo from Ruby #Alaska as ...,1,send photo ruby alaska smoke wildfire pour school,"[175, 117, 5572, 1693, 168, 68, 2314, 103]",8


In [49]:
# as Tensorflow is incapable to create a tensor dataset based on lists, we need to store all our encoded texts into a single numpy array before creating the tensorflow dataset
# furthermore, we also need to use the pad_sequences to add zero padding at the beginning or at the end of our sequences so they all have equal length

reviews_pad = tf.keras.preprocessing.sequence.pad_sequences(df.text_encoded, padding="post")
full_ds = tf.data.Dataset.from_tensor_slices((reviews_pad, df.target.values))

In [50]:
# train test split (70/30)
n_samples = df.shape[0]
TAKE_SIZE = int(0.7 * n_samples)
BATCH_SIZE = 128

# .shuffle() allows to randomly split the train and the test sets
# reshuffle_each_iteration=False is very important in order to avoid changing the splitting of the dataset between train and test
shuffled_ds = full_ds.shuffle(n_samples, reshuffle_each_iteration=False)

train_data_w = shuffled_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE)
test_data_w = shuffled_ds.skip(TAKE_SIZE).shuffle(n_samples - TAKE_SIZE)

# .batch on both sets to organise them by batches of XX observations
train_data = train_data_w.batch(BATCH_SIZE)
test_data = test_data_w.batch(BATCH_SIZE)

In [51]:
# look at a batch
for text, target in train_data.take(1):
  print(text, target)

tf.Tensor(
[[ 254  526  134 ...    0    0    0]
 [ 303  532  775 ...    0    0    0]
 [1513  671 2733 ...    0    0    0]
 ...
 [1049  365  276 ...    0    0    0]
 [  32   69 2924 ...    0    0    0]
 [1664  499   66 ...    0    0    0]], shape=(128, 25), dtype=int32) tf.Tensor(
[1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 1 0 1 1 0 0
 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 0
 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1], shape=(128,), dtype=int64)


## Classification Modeling

LSTMs (Long Short Term Memory networks)

In [52]:
vocab_size = tokenizer.num_words
# text.shape[1] = df.len_text.max()

model = tf.keras.Sequential(
    [
     # input word embedding layer
     Embedding(vocab_size, 64, input_shape=[df.len_text.max(),], name="embedding"),
     
     # first layer LSTM, maintains the sequential nature
     LSTM(units=64, return_sequences=True),
     
     # second layer LSTM, returns the last output
     LSTM(units=32, return_sequences=False),
     
     # classical dense layer once the data is flat
     Dense(16, activation='relu'),
     Dense(8, activation='relu'),
     
     # dropout layer to prevent from overfitting
     # Dropout(rate=0.7),
     
     # output layer with as many neurons as the number of classes (in our case one so activation function is sigmoid)
     Dense(1, activation="sigmoid", name="output_layer")
    ]
)

In [53]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 64)            640000    
                                                                 
 lstm_4 (LSTM)               (None, 25, 64)            33024     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
 output_layer (Dense)        (None, 1)                 9         
                                                                 
Total params: 686,113
Trainable params: 686,113
Non-tr

In [54]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
    # metrics=['accuracy']
)

In [55]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

model.fit(
    train_data,
    epochs=20,
    validation_data=test_data,
    callbacks=[tensorboard_callback]
)

Epoch 1/20
42/42 [==============================] - 6s 44ms/step - loss: 0.6525 - binary_accuracy: 0.6074 - val_loss: 0.5016 - val_binary_accuracy: 0.7885
Epoch 2/20
42/42 [==============================] - 1s 21ms/step - loss: 0.3761 - binary_accuracy: 0.8493 - val_loss: 0.4818 - val_binary_accuracy: 0.7938
Epoch 3/20
42/42 [==============================] - 1s 21ms/step - loss: 0.2423 - binary_accuracy: 0.9116 - val_loss: 0.5844 - val_binary_accuracy: 0.7749
Epoch 4/20
42/42 [==============================] - 1s 22ms/step - loss: 0.1686 - binary_accuracy: 0.9421 - val_loss: 0.6840 - val_binary_accuracy: 0.7696
Epoch 5/20
42/42 [==============================] - 1s 22ms/step - loss: 0.1305 - binary_accuracy: 0.9556 - val_loss: 0.6985 - val_binary_accuracy: 0.7503
Epoch 6/20
42/42 [==============================] - 1s 21ms/step - loss: 0.1043 - binary_accuracy: 0.9678 - val_loss: 0.8664 - val_binary_accuracy: 0.7508
Epoch 7/20
42/42 [==============================] - 1s 21ms/step - los

In [56]:
model.save("/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/model_LSTM.h5")

with open("/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/LSTM_history.json", 'w') as file:
  json.dump(model.history.history, file)

## Classification Evaluation

Visualizing the training process and interpreting the results for our predictive models

In [57]:
with open("/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/LSTM_history.json", 'r') as file:
  LSTM_history = json.load(file)

model_LSTM = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Module 10 - Projects/06 - NLP with Disaster Tweets/model_LSTM.h5")
model_LSTM.summary()
LSTM_history

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 64)            640000    
                                                                 
 lstm_4 (LSTM)               (None, 25, 64)            33024     
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
 output_layer (Dense)        (None, 1)                 9         
                                                                 
Total params: 686,113
Trainable params: 686,113
Non-tr

{'binary_accuracy': [0.6074115633964539,
  0.8493227958679199,
  0.911587655544281,
  0.942061722278595,
  0.9556057453155518,
  0.9678329825401306,
  0.9714070558547974,
  0.9715951681137085,
  0.9766741991043091,
  0.9785553216934204,
  0.9804364442825317,
  0.9810007810592651,
  0.9811888933181763,
  0.9791196584701538,
  0.9802483320236206,
  0.9841986298561096,
  0.9845748543739319,
  0.983258068561554,
  0.9806245565414429,
  0.9834461808204651],
 'loss': [0.652542233467102,
  0.3760858178138733,
  0.24226652085781097,
  0.1685820072889328,
  0.13052453100681305,
  0.10430591553449631,
  0.09316851198673248,
  0.08900918066501617,
  0.07922206073999405,
  0.07444313913583755,
  0.07100298255681992,
  0.06710667163133621,
  0.05975814163684845,
  0.0658387765288353,
  0.058104876428842545,
  0.04414369538426399,
  0.03809509798884392,
  0.04184602573513985,
  0.050385527312755585,
  0.04116816446185112],
 'val_binary_accuracy': [0.7885037064552307,
  0.7937691807746887,
  0.774901

In [58]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(y=LSTM_history["loss"], mode='lines', name='loss'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=LSTM_history["val_loss"], mode='lines', name='val_loss'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=LSTM_history["binary_accuracy"], mode='lines', name='binary_accuracy'),
    # go.Scatter(y=LSTM_history["accuracy"], mode='lines', name='accuracy'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(y=LSTM_history["val_binary_accuracy"], mode='lines', name='val_binary_accuracy'),
    # go.Scatter(y=LSTM_history["val_accuracy"], mode='lines', name='val_accuracy'),
    row=1, col=2
)

fig.update_layout(height=600, width=1200, title_text="Model Performance")
fig.show()